# Vector Search - Query

> Use index create by built-in index feature

Azure AI Search:
- Vector Search
- Hybrid Search
- Semantic Search

https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/readme.md

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

> update value of `index_name` to your index

In [3]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

index_name = 'aoai-docs-idx'

#azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
#azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
#azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]

## Search query

In [4]:
query = "What is OpenAI assistant api?"  
topK = 3

In [5]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

## Text Search

- vector_queries = None
- search_text = query

In [6]:
search_client = SearchClient(endpoint, index_name, credential=credential)
  
# Text Search
results = search_client.search(  
    search_text=query,  
    vector_queries=None,
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")

Score: 6.859043, Title: assistants-studio.md, chunk_id: 2
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMtc3R1ZGlvLm1k0
Score: 6.7678576, Title: assistant-functions.md, chunk_id: 0
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudC1mdW5jdGlvbnMubWQ1
Score: 6.609729, Title: prompt-chat-completion.md, chunk_id: 0
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL3Byb21wdC1jaGF0LWNvbXBsZXRpb24ubWQ1


## Vector Search

- vector_queries = [vector_query]
- search_text = None

In [7]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

# Pure Vector Search
search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")

Score: 0.8802705, Title: assistants.md, chunk_id: 0
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1
Score: 0.86489004, Title: chatgpt-studio.md, chunk_id: 1
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0
Score: 0.86298484, Title: chat-markup-language.md, chunk_id: 5
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0


## Hybrid Search

- vector_queries = [vector query]
- search_text = query

In [8]:
# Hybrid Search
#search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}") 

Score: 0.02982766181230545, Title: chat-markup-language.md, chunk_id: 5
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0
Score: 0.02708333358168602, Title: assistants.md, chunk_id: 0
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1
Score: 0.02194899693131447, Title: chatgpt-studio.md, chunk_id: 1
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0


## Semantic Search (Hybrid search + semantic reranking)

- vector_queries = [vector_query]
- search_text = query
- query_type=QueryType.SEMANTIC,
- semantic_configuration_name='your-semantic-config'

> Update the value of `semantic_configuration_name`

In [9]:
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

# Semantic search
#search_client = SearchClient(endpoint, index_name, credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='aoai-docs-idx-semantic-configuration',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=topK
)

for result in results:
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   Reranker Score: {result['@search.reranker_score']}")
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"   >Caption: {caption.highlights}\n")
        else:
            print(f"   >Caption: {caption.text}\n")
            
semantic_answers = results.get_answers()
if semantic_answers:
    for answer in semantic_answers:
        if answer.highlights:
            print(f"   >Semantic Answer: {answer.highlights}")
        else:
            print(f"   >Semantic Answer: {answer.text}")
        print(f"   >Semantic Answer Score: {answer.score}\n")

Score: 0.02708333358168602, Title: assistants.md, chunk_id: 0
   Reranker Score: 3.403339385986328
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1
   >Caption: <em>Assistants API</em> makes it easier for developers to create applications with sophisticated copilot-like experiences that can sift through data, suggest solutions, and automate tasks. ## Overview  Previously, building custom<em> AI assistants</em> needed heavy lifting even for experienced developers.

Score: 0.012500000186264515, Title: assistants-reference.md, chunk_id: 0
   Reranker Score: 3.0249693393707275
   parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMtcmVmZXJlbmNlLm1k0
   >Caption: assistants-reference.md. --- title: azure<em> openai</em> service<em> assistants</em> python & rest<em> api</em> reference titlesuffix: azure<em> openai</em> description: learn how to use azure<em> openai's</em> python & rest<em> api</em> with assistant